In [1]:
from pathlib import Path

import pandas as pd
pd.options.mode.chained_assignment = None
import numpy as np

import numbers

from sklearn.ensemble import ExtraTreesClassifier, RandomForestClassifier, GradientBoostingClassifier, AdaBoostClassifier
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression, RidgeClassifier, SGDClassifier

import tensorflow as tf
from tensorflow import keras

In [2]:
INITIAL_PATH = Path.cwd().parent
DATA_PATH = INITIAL_PATH / 'data'
PREPROCESSED_DATA_PATH = DATA_PATH / 'preprocessed'
RESULTS_DATA_PATH = INITIAL_PATH / 'results'
SPLITTED_DATA_PATH = PREPROCESSED_DATA_PATH / 'splitted'

In [38]:
PREPROCESSED_KLEKOTA_ROTH_DATA = PREPROCESSED_DATA_PATH / 'cardiotoxicity_hERG_KlekFP.csv'
PREPROCESSED_MACCS_DATA = PREPROCESSED_DATA_PATH / 'cardiotoxicity_hERG_MACCSFP.csv'
PREPROCESSED_EXT_DATA = PREPROCESSED_DATA_PATH / 'cardiotoxicity_hERG_ExtFP.csv'
PREPROCESSED_KLEKOTA_ROTH__MACCS_DATA = PREPROCESSED_DATA_PATH / 'cardiotoxicity_hERG_KlekFP-MACCSFP.csv'
PREPROCESSED_KLEKOTA_ROTH__EXT_DATA = PREPROCESSED_DATA_PATH / 'cardiotoxicity_hERG_KlekFP-ExtFP.csv'
PREPROCESSED_MACCS__EXT_DATA = PREPROCESSED_DATA_PATH / 'cardiotoxicity_hERG_MACCSFP-ExtFP.csv'
PREPROCESSED_KLEKOTA_ROTH__MACCS__EXT_DATA = PREPROCESSED_DATA_PATH / 'cardiotoxicity_hERG_KlekFP-MACCSFP-ExtFP.csv'

RESULTS_KLEKOTA_ROTH = RESULTS_DATA_PATH / 'KlekFP.csv'
RESULTS_MACCS = RESULTS_DATA_PATH / 'MACCSFP.csv'
RESULTS_EXT = RESULTS_DATA_PATH / 'ExtFP.csv'
RESULTS_KLEKOTA_ROTH__MACCS_DATA = RESULTS_DATA_PATH / 'KlekFP-MACCSFP.csv'
RESULTS_KLEKOTA_ROTH__EXT_DATA = RESULTS_DATA_PATH / 'KlekFP-ExtFP.csv'
RESULTS_MACCS__EXT_DATA = RESULTS_DATA_PATH / 'MACCSFP-ExtFP.csv'
RESULTS_KLEKOTA_ROTH__MACCS__EXT_DATA = RESULTS_DATA_PATH / 'KlekFP-MACCSFP-ExtFP.csv'

RESULTS_DEEP_NEURAL_NETWORKS = RESULTS_DATA_PATH / 'deep_neural_networks' / 'results.csv'

In [4]:
PREPROCESSED_KLEKOTA_ROTH_DATA_X = SPLITTED_DATA_PATH / '{}/cardiotoxicity_hERG_KlekFP[{}].csv'
PREPROCESSED_MACCS_DATA_X = SPLITTED_DATA_PATH / '{}/cardiotoxicity_hERG_MACCSFP[{}].csv'
PREPROCESSED_EXT_DATA_X = SPLITTED_DATA_PATH / '{}/cardiotoxicity_hERG_ExtFP[{}].csv'
PREPROCESSED_KLEKOTA_ROTH__MACCS_DATA_X = SPLITTED_DATA_PATH / '{}/cardiotoxicity_hERG_KlekFP-MACCSFP[{}].csv'
PREPROCESSED_KLEKOTA_ROTH__EXT_DATA_X = SPLITTED_DATA_PATH / '{}/cardiotoxicity_hERG_KlekFP-ExtFP[{}].csv'
PREPROCESSED_MACCS__EXT_DATA_X = SPLITTED_DATA_PATH / '{}/cardiotoxicity_hERG_MACCSFP-ExtFP[{}].csv'
PREPROCESSED_KLEKOTA_ROTH__MACCS__EXT_DATA_X = SPLITTED_DATA_PATH / '{}/cardiotoxicity_hERG_KlekFP-MACCSFP-ExtFP.csv[{}]'

RESULTS_KLEKOTA_ROTH_X = RESULTS_DATA_PATH / '{}/KlekFP[{}].csv'
RESULTS_MACCS_X = RESULTS_DATA_PATH / '{}/MACCSFP[{}].csv'
RESULTS_EXT_X = RESULTS_DATA_PATH / '{}/ExtFP[{}].csv'
RESULTS_KLEKOTA_ROTH__MACCS_DATA_X = RESULTS_DATA_PATH / '{}/KlekFP-MACCSFP[{}].csv'
RESULTS_KLEKOTA_ROTH__EXT_DATA_X = RESULTS_DATA_PATH / '{}/KlekFP-ExtFP[{}].csv'
RESULTS_MACCS__EXT_DATA_X = RESULTS_DATA_PATH / '{}/MACCSFP-ExtFP[{}].csv'
RESULTS_KLEKOTA_ROTH__MACCS__EXT_DATA_X = RESULTS_DATA_PATH / '{}/KlekFP-MACCSFP-ExtFP[{}].csv'

In [5]:
class EstimatorSelectionHelper:

    def __init__(self, models, params):
        if not set(models.keys()).issubset(set(params.keys())):
            missing_params = list(set(models.keys()) - set(params.keys()))
            raise ValueError("Some estimators are missing parameters: %s" % missing_params)
        self.models = models
        self.params = params
        self.keys = models.keys()
        self.grid_searches = {}

    def fit(self, X, y, cv=3, n_jobs=3, verbose=1, scoring=None, refit=False):
        for key in self.keys:
            print(f"Running GridSearchCV for {key} with params {self.params[key]} with number of features {len(X.columns)}")
            model = self.models[key]
            params = self.params[key]
            gs = GridSearchCV(model, params, cv=cv, n_jobs=n_jobs,
                              verbose=verbose, scoring=scoring, refit=refit,
                              return_train_score=True)
            gs.fit(X,y)
            self.grid_searches[key] = gs    

    def score_summary(self, sort_by='mean_score'):
        def row(key, scores, params):
            d = {
                 'estimator': key,
                 'min_score': min(scores),
                 'max_score': max(scores),
                 'mean_score': np.mean(scores),
                 'std_score': np.std(scores),
            }
            return pd.Series({**params,**d})

        rows = []
        for k in self.grid_searches:
            params = self.grid_searches[k].cv_results_['params']
            scores = []
            for i in range(self.grid_searches[k].cv):
                key = "split{}_test_score".format(i)
                r = self.grid_searches[k].cv_results_[key]        
                scores.append(r.reshape(len(params),1))

            all_scores = np.hstack(scores)
            for p, s in zip(params,all_scores):
                rows.append((row(k, s, p)))

        df = pd.concat(rows, axis=1).T.sort_values([sort_by], ascending=False)

        columns = ['estimator', 'min_score', 'mean_score', 'max_score', 'std_score']
        columns = columns + [c for c in df.columns if c not in columns]

        return df[columns]

In [6]:
class HyperparametersTuner():
    
    def apply(self, input_filepath, output_filepath):
        df = pd.read_csv(input_filepath)
    
        X = df.drop(['IC50','toxic'], axis=1)
        y = df['toxic']
        
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=123)
        
        print(f'Running Hyperparameters Tunning for {input_filepath}')
        helper = EstimatorSelectionHelper(self.__prepare_models(), self.__prepare_parameters())
        helper.fit(X_train, y_train, scoring='f1', n_jobs=4, verbose=2)
        
        score_summary = helper.score_summary()
        
        score_summary.to_csv(output_filepath, index = False)
        
        return score_summary
    
    @staticmethod
    def __prepare_models():
        return {
            'LogisticRegression': LogisticRegression(), 
            'RidgeClassifier': RidgeClassifier(), 
            'SGDClassifier': SGDClassifier(), 
            'SVC': SVC(),
            'ExtraTreesClassifier': ExtraTreesClassifier(),
            'RandomForestClassifier': RandomForestClassifier(),
            'AdaBoostClassifier': AdaBoostClassifier(),
            'GradientBoostingClassifier': GradientBoostingClassifier(),
        }
    
    @staticmethod
    def __prepare_parameters():
        return {
            'LogisticRegression': {
                'C': [0.01, 0.1, 1, 10],
                'penalty': ['l1', 'l2'],
                'max_iter': list(range(100,500,100)),
                'solver': ['newton-cg', 'lbfgs', 'liblinear', 'sag', 'saga']
            },
            'RidgeClassifier': {
                'alpha': [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0]
            },
            'SGDClassifier': {
                'alpha': [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0],
                'penalty': ['l1', 'l2', 'elasticnet'],
                'max_iter': list(range(100,500,100)),
                'loss': ['hinge', 'log', 'modified_huber', 'squared_hinge', 'perceptron']
            },
            'SVC': {
                'C': [0.01, 0.1, 1, 10],
                'kernel': ['linear', 'poly', 'rbf', 'sigmoid']
            },
            'ExtraTreesClassifier': {
                'n_estimators': list(range(10, 250, 50)),
                'criterion': ['gini', 'entropy'],
                'max_features': ['sqrt', 'log2']
            },
            'RandomForestClassifier': { 
                'n_estimators': list(range(10, 250, 50)),
            },
            'AdaBoostClassifier':  { 
                'n_estimators': list(range(10, 250, 50)),
            },
            'GradientBoostingClassifier': { 
                'n_estimators': list(range(10, 250, 50)),
                'learning_rate': [0.4, 0.6, 0.8, 1.0] 
            }
        }

In [ ]:
data_to_process = [
    (PREPROCESSED_KLEKOTA_ROTH_DATA, RESULTS_KLEKOTA_ROTH), 
    (PREPROCESSED_MACCS_DATA, RESULTS_MACCS), 
    (PREPROCESSED_EXT_DATA, RESULTS_EXT),
    
    (PREPROCESSED_KLEKOTA_ROTH__MACCS_DATA, RESULTS_KLEKOTA_ROTH__MACCS_DATA),
    (PREPROCESSED_KLEKOTA_ROTH__EXT_DATA, RESULTS_KLEKOTA_ROTH__EXT_DATA),
    (PREPROCESSED_MACCS__EXT_DATA, RESULTS_MACCS__EXT_DATA),
    (PREPROCESSED_KLEKOTA_ROTH__MACCS__EXT_DATA, RESULTS_KLEKOTA_ROTH__MACCS__EXT_DATA)
]

hyperparameters_tuner = HyperparametersTuner()
results = []

for data in data_to_process:
    result = hyperparameters_tuner.apply(data[0], data[1])
    results.append(result)

Running Hyperparameters Tunning for C:\Users\SG0306249\Downloads\cardiotoxicity_prediction-main\cardiotoxicity_prediction-main\data\preprocessed\cardiotoxicity_hERG_KlekFP.csv
Running GridSearchCV for LogisticRegression with params {'C': [0.01, 0.1, 1, 10], 'penalty': ['l1', 'l2'], 'max_iter': [100, 200, 300, 400], 'solver': ['newton-cg', 'lbfgs', 'liblinear', 'sag', 'saga']} with number of features 2100
Fitting 3 folds for each of 160 candidates, totalling 480 fits


In [7]:
data_to_process = [
    (PREPROCESSED_KLEKOTA_ROTH_DATA_X, RESULTS_KLEKOTA_ROTH_X), 
    (PREPROCESSED_MACCS_DATA_X, RESULTS_MACCS_X), 
    (PREPROCESSED_EXT_DATA_X, RESULTS_EXT_X),
    
    (PREPROCESSED_KLEKOTA_ROTH__MACCS_DATA_X, RESULTS_KLEKOTA_ROTH__MACCS_DATA_X),
    (PREPROCESSED_KLEKOTA_ROTH__EXT_DATA_X, RESULTS_KLEKOTA_ROTH__EXT_DATA_X),
    (PREPROCESSED_MACCS__EXT_DATA_X, RESULTS_MACCS__EXT_DATA_X),
    (PREPROCESSED_KLEKOTA_ROTH__MACCS__EXT_DATA_X, RESULTS_KLEKOTA_ROTH__MACCS__EXT_DATA_X)
]

hyperparameters_tuner = HyperparametersTuner()
results = []

for x in range(500, 2500, 500):
    for data in data_to_process:
        input_path = Path(str(data[0]).format(x, x))
        output_path = Path(str(data[1]).format(x, x))
        try:
            result = hyperparameters_tuner.apply(input_path, output_path)
            results.append(result)
        except:
            print(f'Ignoring {input_path} because the file does not exists')

Running Hyperparameters Tunning for C:\Users\SG0306249\PycharmProjects\cardiotoxicity_prediction\data\preprocessed\splitted\2000\cardiotoxicity_hERG_KlekFP-MACCSFP[2000].csv
Running GridSearchCV for LogisticRegression with params {'C': [0.01, 0.1, 1, 10], 'penalty': ['l1', 'l2'], 'max_iter': [100, 200, 300, 400], 'solver': ['newton-cg', 'lbfgs', 'liblinear', 'sag', 'saga']} with number of features 2000
Fitting 3 folds for each of 160 candidates, totalling 480 fits


C:\Users\SG0306249\Miniconda3\envs\cardiotoxicity_prediction\lib\site-packages\sklearn\model_selection\_validation.py:372: FitFailedWarning: 
144 fits failed out of a total of 480.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
48 fits failed with the following error:
Traceback (most recent call last):
  File "C:\Users\SG0306249\Miniconda3\envs\cardiotoxicity_prediction\lib\site-packages\sklearn\model_selection\_validation.py", line 681, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\SG0306249\Miniconda3\envs\cardiotoxicity_prediction\lib\site-packages\sklearn\linear_model\_logistic.py", line 1461, in fit
    solver = _check_solver(self.solver, self.penalty, self.dual)
  File "C:\Users\SG0306249\Min

Running GridSearchCV for RidgeClassifier with params {'alpha': [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0]} with number of features 2000
Fitting 3 folds for each of 10 candidates, totalling 30 fits
Running GridSearchCV for SGDClassifier with params {'alpha': [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0], 'penalty': ['l1', 'l2', 'elasticnet'], 'max_iter': [100, 200, 300, 400], 'loss': ['hinge', 'log', 'modified_huber', 'squared_hinge', 'perceptron']} with number of features 2000
Fitting 3 folds for each of 600 candidates, totalling 1800 fits
Running GridSearchCV for SVC with params {'C': [0.01, 0.1, 1, 10], 'kernel': ['linear', 'poly', 'rbf', 'sigmoid']} with number of features 2000
Fitting 3 folds for each of 16 candidates, totalling 48 fits
Running GridSearchCV for ExtraTreesClassifier with params {'n_estimators': [10, 60, 110, 160, 210], 'criterion': ['gini', 'entropy'], 'max_features': ['sqrt', 'log2']} with number of features 2000
Fitting 3 folds for each of 20 candidat

C:\Users\SG0306249\Miniconda3\envs\cardiotoxicity_prediction\lib\site-packages\sklearn\model_selection\_validation.py:372: FitFailedWarning: 
144 fits failed out of a total of 480.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
48 fits failed with the following error:
Traceback (most recent call last):
  File "C:\Users\SG0306249\Miniconda3\envs\cardiotoxicity_prediction\lib\site-packages\sklearn\model_selection\_validation.py", line 681, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\SG0306249\Miniconda3\envs\cardiotoxicity_prediction\lib\site-packages\sklearn\linear_model\_logistic.py", line 1461, in fit
    solver = _check_solver(self.solver, self.penalty, self.dual)
  File "C:\Users\SG0306249\Min

Running GridSearchCV for RidgeClassifier with params {'alpha': [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0]} with number of features 2000
Fitting 3 folds for each of 10 candidates, totalling 30 fits
Running GridSearchCV for SGDClassifier with params {'alpha': [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0], 'penalty': ['l1', 'l2', 'elasticnet'], 'max_iter': [100, 200, 300, 400], 'loss': ['hinge', 'log', 'modified_huber', 'squared_hinge', 'perceptron']} with number of features 2000
Fitting 3 folds for each of 600 candidates, totalling 1800 fits
Running GridSearchCV for SVC with params {'C': [0.01, 0.1, 1, 10], 'kernel': ['linear', 'poly', 'rbf', 'sigmoid']} with number of features 2000
Fitting 3 folds for each of 16 candidates, totalling 48 fits
Running GridSearchCV for ExtraTreesClassifier with params {'n_estimators': [10, 60, 110, 160, 210], 'criterion': ['gini', 'entropy'], 'max_features': ['sqrt', 'log2']} with number of features 2000
Fitting 3 folds for each of 20 candidat

In [80]:
from sklearn.preprocessing import StandardScaler
import keras.layers as layers

class DeepNeuralNetworkTuner():
    
    def __init__(self):
        self.results = {}
    
    def apply(self, input_file):
        print(f'Running Deep Neural Network Tuner for {input_file}')
        df = pd.read_csv(input_file)
        filename = str(input_file).split('\\')[-1]
        data_type = filename.split('.')[0]
        
        X = df.drop(['IC50','toxic'], axis=1)
        y = df['toxic']
        
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=123)
        
        number_of_columns = len(X_train.columns)

        scaler = StandardScaler().fit(X_train)

        X_train = scaler.transform(X_train)
        X_test = scaler.transform(X_test) 
        
        models = {
            "Model_1": keras.Sequential([
                keras.layers.Dense(128, input_shape=(number_of_columns,), kernel_regularizer=keras.regularizers.l1(0.005), activation='relu'),
                keras.layers.Dense(16, activation=tf.nn.relu, kernel_regularizer=keras.regularizers.l2(0.001)),
                keras.layers.Dense(1, activation=tf.nn.sigmoid)
            ]),
            "Model_2": keras.Sequential([
                keras.layers.Dense(128, input_shape=(number_of_columns,), kernel_regularizer=keras.regularizers.l1(0.005), activation='relu'),
                keras.layers.Dense(256, kernel_regularizer=keras.regularizers.l1(0.005), activation='relu'),
                keras.layers.Dense(256, kernel_regularizer=keras.regularizers.l1(0.005), activation='relu'),
                keras.layers.Dense(512, kernel_regularizer=keras.regularizers.l1(0.005), activation='relu'),
                keras.layers.Dense(1, activation=tf.nn.sigmoid)
            ]),
            "Model_3": keras.Sequential([
                keras.layers.Flatten(input_shape=(number_of_columns,)),
                keras.layers.Dense(16, activation=tf.nn.relu, kernel_regularizer=keras.regularizers.l2(0.001)),
                keras.layers.Dense(1, activation=tf.nn.sigmoid),
            ]),
            "Model_4": keras.Sequential([
                keras.layers.Flatten(input_shape=(number_of_columns,)),
                keras.layers.Dense(32, activation=tf.nn.relu, kernel_regularizer=keras.regularizers.l2(0.001)),
                keras.layers.Dense(16, activation=tf.nn.relu, kernel_regularizer=keras.regularizers.l2(0.001)),
                keras.layers.Dense(1, activation=tf.nn.sigmoid),
            ]),
            "Model_5": keras.Sequential([
                keras.layers.Flatten(input_shape=(number_of_columns,)),
                keras.layers.Dense(64, kernel_regularizer=keras.regularizers.l1(0.005), activation='relu'),
                keras.layers.Dense(32, activation=tf.nn.relu, kernel_regularizer=keras.regularizers.l2(0.001)),
                keras.layers.Dense(16, activation=tf.nn.relu, kernel_regularizer=keras.regularizers.l2(0.001)),
                keras.layers.Dense(1, activation=tf.nn.sigmoid),
            ])
        }
            
        for model_name, model in models.items():
            model.compile(optimizer='adam',
                          loss='binary_crossentropy',
                          metrics=['accuracy'])

            early_stopping = keras.callbacks.EarlyStopping(monitor='val_accuracy',
                                                           mode='max',
                                                           patience=10,
                                                           restore_best_weights=True)

            print(f'Running {model_name}')
            model.fit(X_train, y_train, 
                      epochs=300, 
                      batch_size=10,
                      callbacks=[early_stopping],
                      validation_split=0.2,
                      shuffle=True)

            test_loss, test_acc = model.evaluate(X_test, y_test)
            print(f'Finished {model_name}, got results: test_loss={test_loss}, test_acc={test_acc}')
            
            self.results[f'{data_type}_{model_name}'] = {
                'test_acc':test_acc,
                'test_loss':test_loss    
            }
    
    def summary(self, output_file):
        sorted_results = {k: v for k, v in sorted(self.results.items(), key=lambda item: item[1]['test_acc'], reverse=True)}
        df = pd.DataFrame(sorted_results)
        df.to_csv(output_file, index = False)

In [82]:
PREPROCESSED_KLEKOTA_ROTH_DATA = PREPROCESSED_DATA_PATH / 'cardiotoxicity_hERG_KlekFP.csv'
PREPROCESSED_MACCS_DATA = PREPROCESSED_DATA_PATH / 'cardiotoxicity_hERG_MACCSFP.csv'
PREPROCESSED_EXT_DATA = PREPROCESSED_DATA_PATH / 'cardiotoxicity_hERG_ExtFP.csv'
PREPROCESSED_KLEKOTA_ROTH__MACCS_DATA = PREPROCESSED_DATA_PATH / 'cardiotoxicity_hERG_KlekFP-MACCSFP.csv'
PREPROCESSED_KLEKOTA_ROTH__EXT_DATA = PREPROCESSED_DATA_PATH / 'cardiotoxicity_hERG_KlekFP-ExtFP.csv'
PREPROCESSED_MACCS__EXT_DATA = PREPROCESSED_DATA_PATH / 'cardiotoxicity_hERG_MACCSFP-ExtFP.csv'
PREPROCESSED_KLEKOTA_ROTH__MACCS__EXT_DATA = PREPROCESSED_DATA_PATH / 'cardiotoxicity_hERG_KlekFP-MACCSFP-ExtFP.csv'

data_to_process = [
    (PREPROCESSED_KLEKOTA_ROTH_DATA,),
    (PREPROCESSED_MACCS_DATA,),
    (PREPROCESSED_EXT_DATA,),
    (PREPROCESSED_KLEKOTA_ROTH__MACCS_DATA,),
    (PREPROCESSED_KLEKOTA_ROTH__EXT_DATA,),
    (PREPROCESSED_MACCS__EXT_DATA,),
    (PREPROCESSED_KLEKOTA_ROTH__MACCS__EXT_DATA,),
]


deepNeuralNetworkTuner = DeepNeuralNetworkTuner()

for data in data_to_process:
    deepNeuralNetworkTuner.apply(data[0])

deepNeuralNetworkTuner.summary(RESULTS_DEEP_NEURAL_NETWORKS)

Running Deep Neural Network Tuner for C:\Users\SG0306249\Downloads\cardiotoxicity_prediction-main\cardiotoxicity_prediction-main\data\preprocessed\cardiotoxicity_hERG_KlekFP.csv
Running Model_1
Epoch 1/300
681/681 [==============================] - 6s 8ms/step - loss: 2.8486 - accuracy: 0.6483 - val_loss: 1.3574 - val_accuracy: 0.6833
Epoch 2/300
681/681 [==============================] - 6s 8ms/step - loss: 1.1894 - accuracy: 0.7056 - val_loss: 1.1088 - val_accuracy: 0.7045
Epoch 3/300
681/681 [==============================] - 5s 8ms/step - loss: 1.0664 - accuracy: 0.7233 - val_loss: 1.0448 - val_accuracy: 0.7027
Epoch 4/300
681/681 [==============================] - 3s 4ms/step - loss: 1.0041 - accuracy: 0.7373 - val_loss: 1.0355 - val_accuracy: 0.7127
Epoch 5/300
681/681 [==============================] - 5s 8ms/step - loss: 0.9889 - accuracy: 0.7442 - val_loss: 1.0434 - val_accuracy: 0.7192
Epoch 6/300
681/681 [==============================] - 4s 6ms/step - loss: 0.9528 - accurac

681/681 [==============================] - 1s 1ms/step - loss: 0.4078 - accuracy: 0.8579 - val_loss: 0.6436 - val_accuracy: 0.7597
Epoch 5/300
681/681 [==============================] - 1s 1ms/step - loss: 0.3898 - accuracy: 0.8700 - val_loss: 0.6606 - val_accuracy: 0.7667
Epoch 6/300
681/681 [==============================] - 1s 1ms/step - loss: 0.3698 - accuracy: 0.8807 - val_loss: 0.6958 - val_accuracy: 0.7591
Epoch 7/300
681/681 [==============================] - 1s 1ms/step - loss: 0.3592 - accuracy: 0.8916 - val_loss: 0.6874 - val_accuracy: 0.7656
Epoch 8/300
681/681 [==============================] - 1s 1ms/step - loss: 0.3512 - accuracy: 0.8994 - val_loss: 0.7072 - val_accuracy: 0.7761
Epoch 9/300
681/681 [==============================] - 1s 1ms/step - loss: 0.3498 - accuracy: 0.9019 - val_loss: 0.7568 - val_accuracy: 0.7679
Epoch 10/300
681/681 [==============================] - 1s 1ms/step - loss: 0.3425 - accuracy: 0.9029 - val_loss: 0.7540 - val_accuracy: 0.7650
Epoch 11/3

67/67 [==============================] - 0s 1ms/step - loss: 0.8652 - accuracy: 0.7330
Finished Model_5, got results: test_loss=0.8651915788650513, test_acc=0.7329572439193726
Running Deep Neural Network Tuner for C:\Users\SG0306249\Downloads\cardiotoxicity_prediction-main\cardiotoxicity_prediction-main\data\preprocessed\cardiotoxicity_hERG_MACCSFP.csv
Running Model_1
Epoch 1/300
681/681 [==============================] - 1s 1ms/step - loss: 1.8269 - accuracy: 0.6502 - val_loss: 0.7611 - val_accuracy: 0.6798
Epoch 2/300
681/681 [==============================] - 1s 921us/step - loss: 0.7289 - accuracy: 0.6815 - val_loss: 0.6955 - val_accuracy: 0.6880
Epoch 3/300
681/681 [==============================] - 1s 936us/step - loss: 0.6816 - accuracy: 0.7014 - val_loss: 0.6779 - val_accuracy: 0.6992
Epoch 4/300
681/681 [==============================] - 1s 917us/step - loss: 0.6629 - accuracy: 0.7004 - val_loss: 0.6600 - val_accuracy: 0.7115
Epoch 5/300
681/681 [==============================

681/681 [==============================] - 0s 639us/step - loss: 0.4399 - accuracy: 0.8088 - val_loss: 0.5732 - val_accuracy: 0.7479
Epoch 17/300
681/681 [==============================] - 0s 628us/step - loss: 0.4360 - accuracy: 0.8112 - val_loss: 0.5769 - val_accuracy: 0.7450
Epoch 18/300
681/681 [==============================] - 0s 615us/step - loss: 0.4340 - accuracy: 0.8140 - val_loss: 0.5688 - val_accuracy: 0.7509
Epoch 19/300
681/681 [==============================] - 0s 633us/step - loss: 0.4288 - accuracy: 0.8187 - val_loss: 0.5748 - val_accuracy: 0.7474
Epoch 20/300
681/681 [==============================] - 0s 614us/step - loss: 0.4265 - accuracy: 0.8169 - val_loss: 0.5862 - val_accuracy: 0.7397
Epoch 21/300
681/681 [==============================] - 0s 625us/step - loss: 0.4278 - accuracy: 0.8197 - val_loss: 0.5875 - val_accuracy: 0.7444
Epoch 22/300
681/681 [==============================] - 0s 617us/step - loss: 0.4217 - accuracy: 0.8210 - val_loss: 0.5833 - val_accuracy

681/681 [==============================] - 1s 1ms/step - loss: 0.7925 - accuracy: 0.7133 - val_loss: 0.8518 - val_accuracy: 0.6874
Epoch 9/300
681/681 [==============================] - 1s 1ms/step - loss: 0.7846 - accuracy: 0.7204 - val_loss: 0.8127 - val_accuracy: 0.6974
Epoch 10/300
681/681 [==============================] - 1s 1ms/step - loss: 0.7852 - accuracy: 0.7273 - val_loss: 0.8019 - val_accuracy: 0.6933
Epoch 11/300
681/681 [==============================] - 1s 1ms/step - loss: 0.7770 - accuracy: 0.7208 - val_loss: 0.8030 - val_accuracy: 0.7039
Epoch 12/300
681/681 [==============================] - 1s 1ms/step - loss: 0.7595 - accuracy: 0.7258 - val_loss: 0.7914 - val_accuracy: 0.6998
Epoch 13/300
681/681 [==============================] - 1s 1ms/step - loss: 0.7645 - accuracy: 0.7247 - val_loss: 0.8068 - val_accuracy: 0.7062
Epoch 14/300
681/681 [==============================] - ETA: 0s - loss: 0.7672 - accuracy: 0.72 - 1s 1ms/step - loss: 0.7665 - accuracy: 0.7294 - val_

...

681/681 [==============================] - 2s 3ms/step - loss: 0.8739 - accuracy: 0.7461 - val_loss: 0.9170 - val_accuracy: 0.7074
Epoch 17/300
681/681 [==============================] - 2s 3ms/step - loss: 0.8749 - accuracy: 0.7443 - val_loss: 0.8968 - val_accuracy: 0.7192
Epoch 18/300
681/681 [==============================] - 2s 3ms/step - loss: 0.8760 - accuracy: 0.7496 - val_loss: 0.9520 - val_accuracy: 0.7092
Epoch 19/300
681/681 [==============================] - 2s 3ms/step - loss: 0.8680 - accuracy: 0.7571 - val_loss: 0.8991 - val_accuracy: 0.7315
Epoch 20/300
681/681 [==============================] - 2s 3ms/step - loss: 0.8725 - accuracy: 0.7515 - val_loss: 0.9227 - val_accuracy: 0.7209
Epoch 21/300
681/681 [==============================] - 2s 3ms/step - loss: 0.8706 - accuracy: 0.7567 - val_loss: 0.8940 - val_accuracy: 0.7297
Epoch 22/300
681/681 [==============================] - 2s 2ms/step - loss: 0.8681 - accuracy: 0.7530 - val_loss: 0.9248 - val_accuracy: 0.7162
Epoch

681/681 [==============================] - 1s 1ms/step - loss: 0.7339 - accuracy: 0.7521 - val_loss: 0.7776 - val_accuracy: 0.7280
Epoch 38/300
681/681 [==============================] - 1s 1ms/step - loss: 0.7308 - accuracy: 0.7476 - val_loss: 0.7655 - val_accuracy: 0.7397
Epoch 39/300
681/681 [==============================] - 1s 1ms/step - loss: 0.7322 - accuracy: 0.7524 - val_loss: 0.8598 - val_accuracy: 0.7262
Epoch 40/300
681/681 [==============================] - 1s 1ms/step - loss: 0.7375 - accuracy: 0.7467 - val_loss: 0.7785 - val_accuracy: 0.7256
Epoch 41/300
681/681 [==============================] - 1s 1ms/step - loss: 0.7268 - accuracy: 0.7530 - val_loss: 0.7791 - val_accuracy: 0.7268
Epoch 42/300
681/681 [==============================] - 1s 1ms/step - loss: 0.7302 - accuracy: 0.7517 - val_loss: 0.7666 - val_accuracy: 0.7203
Epoch 43/300
681/681 [==============================] - 1s 1ms/step - loss: 0.7239 - accuracy: 0.7524 - val_loss: 0.7754 - val_accuracy: 0.7244
Epoch

Epoch 8/300
681/681 [==============================] - 2s 2ms/step - loss: 0.3872 - accuracy: 0.8828 - val_loss: 0.6574 - val_accuracy: 0.7662
Epoch 9/300
681/681 [==============================] - 1s 2ms/step - loss: 0.3860 - accuracy: 0.8852 - val_loss: 0.6802 - val_accuracy: 0.7615
Epoch 10/300
681/681 [==============================] - 1s 2ms/step - loss: 0.3744 - accuracy: 0.8930 - val_loss: 0.6909 - val_accuracy: 0.7709
Epoch 11/300
681/681 [==============================] - 1s 2ms/step - loss: 0.3601 - accuracy: 0.8982 - val_loss: 0.7335 - val_accuracy: 0.7626
Epoch 12/300
681/681 [==============================] - 1s 2ms/step - loss: 0.3554 - accuracy: 0.9019 - val_loss: 0.7002 - val_accuracy: 0.7703
Epoch 13/300
681/681 [==============================] - 1s 1ms/step - loss: 0.3575 - accuracy: 0.9023 - val_loss: 0.7186 - val_accuracy: 0.7626
Epoch 14/300
681/681 [==============================] - 1s 2ms/step - loss: 0.3540 - accuracy: 0.9049 - val_loss: 0.7000 - val_accuracy: 0

681/681 [==============================] - 1s 1ms/step - loss: 0.6879 - accuracy: 0.7455 - val_loss: 0.7204 - val_accuracy: 0.7250
Epoch 40/300
67/67 [==============================] - 0s 917us/step - loss: 0.7325 - accuracy: 0.7076
Finished Model_5, got results: test_loss=0.7325151562690735, test_acc=0.7075693607330322
Running Deep Neural Network Tuner for C:\Users\SG0306249\Downloads\cardiotoxicity_prediction-main\cardiotoxicity_prediction-main\data\preprocessed\cardiotoxicity_hERG_KlekFP-MACCSFP-ExtFP.csv
Running Model_1
Epoch 1/300
681/681 [==============================] - 2s 3ms/step - loss: 3.6507 - accuracy: 0.6560 - val_loss: 1.6180 - val_accuracy: 0.6675
Epoch 2/300
681/681 [==============================] - 2s 3ms/step - loss: 1.3601 - accuracy: 0.7094 - val_loss: 1.2351 - val_accuracy: 0.7209
Epoch 3/300
681/681 [==============================] - 2s 3ms/step - loss: 1.2030 - accuracy: 0.7086 - val_loss: 1.1748 - val_accuracy: 0.7092
Epoch 4/300
681/681 [====================

Epoch 17/300
681/681 [==============================] - 1s 1ms/step - loss: 0.2809 - accuracy: 0.9305 - val_loss: 0.7583 - val_accuracy: 0.7908
Epoch 18/300
681/681 [==============================] - 1s 1ms/step - loss: 0.2781 - accuracy: 0.9301 - val_loss: 0.7745 - val_accuracy: 0.7838
Epoch 19/300
681/681 [==============================] - 1s 1ms/step - loss: 0.2760 - accuracy: 0.9292 - val_loss: 0.7953 - val_accuracy: 0.7820
Epoch 20/300
681/681 [==============================] - 1s 1ms/step - loss: 0.2809 - accuracy: 0.9345 - val_loss: 0.7807 - val_accuracy: 0.7861
Epoch 21/300
681/681 [==============================] - 1s 1ms/step - loss: 0.2698 - accuracy: 0.9337 - val_loss: 0.8131 - val_accuracy: 0.7697
Epoch 22/300
681/681 [==============================] - 1s 1ms/step - loss: 0.2614 - accuracy: 0.9370 - val_loss: 0.8678 - val_accuracy: 0.7656
Epoch 23/300
681/681 [==============================] - 1s 1ms/step - loss: 0.2681 - accuracy: 0.9324 - val_loss: 0.8236 - val_accuracy: